<a href="https://colab.research.google.com/github/Sakha-Language-Processing/wordforms/blob/main/verbs_at_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Автоматизированный поиск глаголов как последних слов в предложении



Из 541024 предложений выделены последние слова, которые с большой вероятностью являются глаголами. Получилось 20408 словоформ. Доля глаголов является высокой.

Около 80% текста потеряно.

## Загрузка и очистка 

In [9]:
!wget https://github.com/Sakha-Language-Processing/parser-palament/raw/main/raw_data_parlamentsakha.zip
!unzip raw_data_parlamentsakha.zip
!wget https://github.com/Sakha-Language-Processing/parser-nvkonline/raw/main/raw_data_nvkonline.zip
!unzip raw_data_nvkonline.zip
!wget https://github.com/Sakha-Language-Processing/sakha-parser-keskil/raw/main/raw_data_keskil_fixed.zip
!unzip raw_data_keskil_fixed.zip
!wget https://github.com/Sakha-Language-Processing/parser-kyym/raw/main/kyym-df.zip
!unzip kyym-df.zip
!wget https://github.com/Sakha-Language-Processing/parser-ysia/raw/main/sakha_ysia_df.zip
!unzip sakha_ysia_df.zip

--2021-04-18 13:28:10--  https://github.com/Sakha-Language-Processing/parser-palament/raw/main/raw_data_parlamentsakha.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/Sakha-Language-Processing/parser-parlament/raw/main/raw_data_parlamentsakha.zip [following]
--2021-04-18 13:28:10--  https://github.com/Sakha-Language-Processing/parser-parlament/raw/main/raw_data_parlamentsakha.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Sakha-Language-Processing/parser-parlament/main/raw_data_parlamentsakha.zip [following]
--2021-04-18 13:28:10--  https://raw.githubusercontent.com/Sakha-Language-Processing/parser-parlament/main/raw_data_parlamentsakha.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 18

In [37]:
# Чтение файлов
# Можно добавить свои список файлов 
import json
import pandas as pd

articles = []
for file in ['raw_data_nvkonline.json',
             'raw_data_parlamentsakha.json',
             'raw_data_keskil.json']:
    file_ = open(file) 
    articles  += json.load(file_)
    file_.close()

all_files_text = []
for article in articles:
    all_files_text += article['raw_text'] 
    if 'title' in article:
        all_files_text += [article['title']]

aytal_text = ' '
for file in ['kyym-novosti.csv',
             'sakha_ysia.csv']:
    file_ = pd.read_csv(file) 
    aytal_text  += ' '.join(list(file_['title'].fillna(' ')+file_['all_text'].fillna(' ')))

all_files_text = ' '.join(all_files_text) + ' '.join(aytal_text)
del aytal_text

In [38]:
textslist = all_files_text.split('  ')
all_files_text = ' '.join([item for item in textslist if len(item)>1])
all_files_text += ' '.join([item for item in textslist if len(item)==1])


In [39]:
#  в некоторых статьях буквы стоят отдельно

tezt = all_files_text.replace(' ','')
# print(n)
# print(all_files_text[n-100:n+300])


# print('Имеется', len([item for item in all_files_text.split(' ')  if len(item)>1]), 'слов.')
# x1,x2 = all_files_text[:100000].count('ы'),all_files_text.count(' ы ')
# x_slov = len(all_files_text[:100000].split(' '))
# print('Потеряно примерно', int(x_slov/x1*x2), 'слов.')


In [40]:
len(tezt.split(' '))

1

In [133]:
n = 14245000
all_files_text[n-300:n+300]


'т .   К и н и   м а н н а   б и и р   д о й д у л а а х т а р ы н   к ы т а р ы   к ө р с ү б ү т .   К о н с у л   т о й о н   Т а р а с е н к о н ы   У з б е к и с т а ҥ ҥ а   ы а л д ь ы т т а т а   ы ҥ ы р б ы т .   Х а а р т ы с к а н ы   М а р и я   В а с и л ь е в а ,   С И А   т ү һ э р б и т .     Э Л И М И Э Н     Э Р Э Й   Д У У ,   Э Р Э Л   Д У У ?   А ҕ ы й а х   с ы л л а а ҕ ы т а   А н а а б ы р г а   ү л э л и и   с ы л д ь а н ,   б и и р   у л а х а н   у ч а а с т а к к а   ү л э б и т   т ү б ү г э р   с ы л д ь а н   б а р а м м ы т ,   б э й э б и т   к ы р а к ы й   у '

In [45]:
import re
all_files_text = re.sub(re.compile(r'https?\S+'), ' ', all_files_text)
all_files_text = re.sub(re.compile(r'<.*?>'), ' ', all_files_text)
all_files_text = re.sub(re.compile(r'@\S+'), ' ', all_files_text)
all_files_text = re.sub(re.compile(r'[\//:@&\-\'\`\"\_\n\#]'), ' ', all_files_text)
all_files_text = re.sub(re.compile(r'\s*[A-Za-z\d.]+\b'), ' ' , all_files_text)
all_files_text = all_files_text.lower()
similar_characters = {'h':'һ','c':'с','x':'х','y':'у',
                      'e':'е', 'T':'Т', 'o':'о', 'p':'р', 
                      'a':'а', 'k':'к', 'b':'в', 'm':'м', 
                      '5':'ҕ', '8':'ө',
                      # исправления
                      'ааа':'аа','ттт':'тт','ыыы':'ыы',
                      'ууу':'уу','   ':' ','  ':' ',
                      'иии':'ии','эээ':'ээ','үүү':'үү',
                      'өөө':'өө','өү':'үө','ччч':'чч',
                      'ннн':'нн','ддд':'дд','ллл':'лл',    
                      'ммм':'мм','ххх':'хх',    
                      }                      
for loaned, ersatz in similar_characters.items():
    all_files_text = all_files_text.replace(loaned, ersatz)  

all_files_text = re.sub(re.compile(r'[.!?]'), ' ', all_files_text)
all_files_text = all_files_text.replace('      ',' ')
all_files_text = all_files_text.replace('   ',' ')
all_files_text = all_files_text.replace('  ',' ')
all_files_text = all_files_text.replace('  ',' ')
all_files_text.count('  ')



In [62]:
ddd = [wordform.strip() for wordform in all_files_text.split(' ') if len(wordform)>1]
ddd = sorted(ddd)

words = []
counts = []
counter = 0
for item1,item2 in zip(ddd[:-1],ddd[1:]):
    counter += 1
    if item1!=item2:
       words.append(item1)
       counts.append(counter)
       counter = 0
df = pd.DataFrame({'wordform':words,'occurence':counts})          

## Выделение последних слов предложений

In [13]:

all_files_text = all_files_text.replace('?', '.')
all_files_text = all_files_text.replace('!', '.')
regex = re.compile('[^а-яА-ЯһөҕүҥҺӨҔҮҤ. ]')
all_files_text = re.sub(regex, ' ', all_files_text)

sentence_tails = [item[-min(len(item),30):] for item in all_files_text.split('.')]

verbs = []
for item in sentence_tails:
    verb = item.split(' ')[-1]
    if len(verb)<1:
         for i in range(30):
            item = item.replace(' .','.')
            verb = item.split(' ')[-1]
    verbs.append(verb) 
verbs = sorted(list(set(verbs)))
df = pd.DataFrame(verbs)
df.columns = ['verb']
df.to_csv('verbs.csv',index=False)

In [14]:
len(sentence_tails)


541024